In [4]:
import pandas as pd
import numpy as np
from collections import Counter
import csv
import jieba
import re
import matplotlib.pyplot as plt
import seaborn as sns
from keras import Sequential
import keras_bert 
from keras import models
from keras import Sequential
from keras import layers
from keras.layers import LSTM,Embedding,Dense,GRU,SimpleRNN
from keras.utils import to_categorical

In [5]:
#读取李白诗词数据集
libai=pd.read_csv(r'C:\Users\Administrator\Desktop\李白诗词.txt',encoding='utf-8')
#得到李白诗词数据列表
poetry_lis=[''.join(i.split()) for i in libai['      李白诗词      ']]


In [6]:
def poetry_optimize(poetry_lis:list):
    '''并去除<p>卷、「诗名进行去除。将每行诗句以逗号、句号分隔开，'''
    new_poetrycontnt=[]
    for every_poetry in poetry_lis:
        poetry_content=every_poetry.split('<')[0]
        if '「'  not in poetry_content:
            new_poetrycontnt.append(''.join(re.split('[，。]',poetry_content)))
    return new_poetrycontnt

#jieba分词
def get_worddict(new_poetrycontnt:list):
    '''将列表中的每一个诗句进行分词，使用jieba分词得到全部词，形成词典
    '''
    Token_poetry_content=[]
    word_dict=dict()
    for sentence in new_poetrycontnt:
       for word in jieba.lcut(sentence,cut_all=True):
            if word not in word_dict:
                word_dict[word]=1
            else:
                word_dict[word]+=1
            Token_poetry_content.append(word)
    return word_dict,Token_poetry_content



In [7]:
#统计词频
#查看词频图
new_poetrycontnt=poetry_optimize(poetry_lis)
word_dict,Token_poetry_content=get_worddict(new_poetrycontnt)
#pd.Series(word_dict).plot.bar()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.980 seconds.
Prefix dict has been built successfully.


In [8]:
#将词映射称数字
def word_to_index(word_dict):
   '''将字映射成字符'''
   word_to_index=dict()
   for index,word in enumerate(word_dict):
      word_to_index[word]=index
   return word_to_index
word_index=word_to_index(word_dict)

In [9]:
#########################
#数据处理
#########将李白诗词进行分词，并拿key进行替代
def poetry_to_sequence(Token_poetry_content,word_index):
    sequence=[]
    for word in Token_poetry_content:
        sequence.append(word_index[word])
    return sequence
        
    

In [10]:
sequence=poetry_to_sequence(Token_poetry_content,word_index)

In [17]:
################滑动窗口
def slide_window(sequence,windowsize=5):
    '''滑动窗口采集数据'''
    feature_data=[]
    label_data=[]
    for _ in range(len(sequence)-windowsize):
        feature,label=[[i] for i in sequence[_:_+windowsize]],sequence[_+windowsize]

        feature_data.append(feature)
        label_data.append(label)
    return feature_data,label_data

In [18]:
feature_data,label_data=slide_window(sequence)
label_data=to_categorical(label_data)
model=models.Sequential()
model.add(layers.BatchNormalization())
model.add(layers.LSTM(32,input_shape=(len(feature_data),1),activation='relu',return_sequences=True))
model.add(layers.BatchNormalization())
model.add(layers.LSTM(16,activation='elu'))
model.add(layers.Dense(10129,activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics='acc')
model.fit(np.array(feature_data),label_data,epochs=5,batch_size=64,validation_data=0.3)

In [109]:
def findmax_index(nums):
    index=0
    max=0
    for index,number in enumerate(nums):
        if number>max:
            max=number
            index=index
    return max,index
def findmax_index(nums):
    maxindex=0
    max=0
    for index,number in enumerate(nums):
        if number>max:
            max=number
            maxindex=index
    return max,maxindex

def get_predict(model,input_word,word_index,maxlen=100):
    for num in range(maxlen): 
        token_input_word=jieba.lcut(input_word)
        token_input_word_to_index=[[word_index[word]] if word in word_index else [np.random.randint(20000)] for word in token_input_word ]
        if len(token_input_word)<5:
            for _ in range(0,5-len(token_input_word_to_index)):
                token_input_word_to_index.append([np.random.randint(20000)])
        predict_lis=model.predict(token_input_word_to_index[-5:])
        maxproba,maxprebawordindex=findmax_index(predict_lis[0])
        index_to_word=dict([(j,i) for i,j in word_index.items()])
        next_word=index_to_word[maxprebawordindex]
        input_word+=next_word
    print(input_word+next_word)


    

In [158]:
input='啊啊啊啊啊'

In [159]:
get_predict(model,input,word_index)

AttributeError: 'Sequential' object has no attribute 'decode'

In [125]:
label_data.shape

(57507, 10129)

In [170]:
def get_randompredict(input_word,word_index,maxlen=20):
    now_nums_word=jieba.lcut(input_word)
    for _ in range(maxlen-len(now_nums_word)):
        new_word_index=np.random.randint(10129)
        index_to_word=dict([(j,i) for i,j in word_index.items()])
        new_word=index_to_word[new_word_index]
        input_word+=new_word
    
    return input_word
    


In [186]:
input_word=get_randompredict(input,word_index,maxlen=20)
input_word

'啊啊啊啊啊蕖阐苦难蹳逵百万明晨久留心乱窗户沅切秦汉珠玉川北时光炼'

In [187]:
def optimize(input_word):
    new_str=[]
    for i in range(len(input_word)):
      if i//5==0:
        new_str.append(input_word[i:i+5])
    return ','.join(new_str)

In [188]:
outputsentence=optimize(str(get_randompredict(input,word_index,maxlen=100)))

In [189]:
outputsentence

'啊啊啊啊啊,啊啊啊啊著,啊啊啊著书,啊啊著书鸿,啊著书鸿沟'

In [171]:
str(get_randompredict(input,word_index,maxlen=100))

'啊啊啊啊啊两客后门枉杀老僧济悸优游昨夜仙郎坚开白葭一经倾国改化作沙伊川崆峒携标猿啼炼气茫户长豕翮残袖长南京涂泥中期河湾不啻交相喷薄残兵苔绿色一往玩忽一枝荆扉夷门下去四争战比颇然后巾东妓女徒劳夕烟洗染古意莫邪重价云山永隐然霹雳逸兴峰嶂右军丰年怡然三边凋落嗽饲寥寥下列苑向海准回向襜脱倾倒鸣凤硾云汉千里名花玉露生遍宝刀写意垂青得免贫病陆相娱长江罗江奔走军威'